# নমুনা ০১: OpenAI SDK ব্যবহার করে দ্রুত চ্যাট

এই নোটবুকটি Microsoft Foundry Local-এর সাথে OpenAI SDK ব্যবহার করে দ্রুত চ্যাট ইন্টারঅ্যাকশন করার পদ্ধতি প্রদর্শন করে।

## সংক্ষিপ্ত বিবরণ

এই উদাহরণটি দেখায়:
- Foundry Local-এর সাথে OpenAI Python SDK ব্যবহার করা
- Azure OpenAI এবং স্থানীয় Foundry কনফিগারেশন উভয়ের পরিচালনা
- সঠিক ত্রুটি পরিচালনা এবং বিকল্প কৌশল বাস্তবায়ন
- সেবা ব্যবস্থাপনার জন্য FoundryLocalManager ব্যবহার করা


## পূর্বশর্ত

নিশ্চিত করুন যে আপনার কাছে আছে:
1. Foundry Local ইনস্টল করা এবং চালু
2. একটি মডেল লোড করা (যেমন, `foundry model run phi-4-mini`)
3. প্রয়োজনীয় Python প্যাকেজগুলো ইনস্টল করা

### নির্ভরতা ইনস্টল করুন


In [ ]:
# Install required packages if not already installed
!pip install openai foundry-local-sdk

## প্রয়োজনীয় লাইব্রেরি আমদানি করুন


In [ ]:
import os
import sys
from openai import OpenAI

try:
    from foundry_local import FoundryLocalManager
    FOUNDRY_SDK_AVAILABLE = True
    print("✅ Foundry Local SDK is available")
except ImportError:
    FOUNDRY_SDK_AVAILABLE = False
    print("⚠️ Foundry Local SDK not available, will use manual configuration")

## কনফিগারেশন

আপনার কনফিগারেশন সেট আপ করুন। আপনি ব্যবহার করতে পারেন:
1. **Azure OpenAI** (ক্লাউড-ভিত্তিক)
2. **Foundry Local** (লোকাল ইনফারেন্স)

### অপশন ১: Azure OpenAI কনফিগারেশন


In [ ]:
# Azure OpenAI Configuration (uncomment and set your values)
# os.environ["AZURE_OPENAI_ENDPOINT"] = "https://your-resource.openai.azure.com"
# os.environ["AZURE_OPENAI_API_KEY"] = "your-api-key"
# os.environ["AZURE_OPENAI_API_VERSION"] = "2024-08-01-preview"
# os.environ["MODEL"] = "your-deployment-name"

### বিকল্প ২: ফাউন্ড্রি স্থানীয় কনফিগারেশন


In [ ]:
# Foundry Local Configuration
MODEL_ALIAS = "phi-4-mini"  # Change this to your preferred model
BASE_URL = "http://localhost:8000"  # Default Foundry Local URL
API_KEY = ""  # Usually not needed for local

## ক্লায়েন্ট আরম্ভ

আপনার কনফিগারেশন অনুযায়ী OpenAI ক্লায়েন্ট তৈরি করুন:


In [ ]:
def create_client():
    """Create and configure OpenAI client."""
    
    # Check for Azure OpenAI configuration
    azure_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
    azure_api_key = os.environ.get("AZURE_OPENAI_API_KEY")
    azure_api_version = os.environ.get("AZURE_OPENAI_API_VERSION", "2024-08-01-preview")
    
    if azure_endpoint and azure_api_key:
        # Azure OpenAI path
        model = os.environ.get("MODEL", "your-deployment-name")
        client = OpenAI(
            base_url=f"{azure_endpoint}/openai",
            api_key=azure_api_key,
            default_query={"api-version": azure_api_version},
        )
        print(f"🌐 Using Azure OpenAI with model: {model}")
        return client, model
    
    # Foundry Local path
    if FOUNDRY_SDK_AVAILABLE:
        try:
            # Use FoundryLocalManager for proper service management
            manager = FoundryLocalManager(MODEL_ALIAS)
            model_info = manager.get_model_info(MODEL_ALIAS)
            
            # Configure OpenAI client to use local Foundry service
            client = OpenAI(
                base_url=manager.endpoint,
                api_key=manager.api_key  # API key is not required for local usage
            )
            model = model_info.id
            print(f"🏠 Using Foundry Local SDK with model: {model}")
            return client, model
        except Exception as e:
            print(f"⚠️ Could not use Foundry SDK ({e}), falling back to manual configuration")
    
    # Fallback to manual configuration
    client = OpenAI(
        base_url=f"{BASE_URL}/v1",
        api_key=API_KEY
    )
    model = MODEL_ALIAS
    print(f"🔧 Using manual configuration with model: {model}")
    return client, model

# Initialize the client
client, model = create_client()

## সাধারণ চ্যাট উদাহরণ

চলুন একটি সহজ চ্যাট ইন্টারঅ্যাকশন পরীক্ষা করি:


In [ ]:
def chat_with_model(prompt, max_tokens=128):
    """Send a chat message to the model and get a response."""
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens
        )
        
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {e}"

# Test with a simple prompt
prompt = "Say hello from Foundry Local and explain what you are in one sentence."
print(f"👤 User: {prompt}")
print("\n🤖 Assistant:")
response = chat_with_model(prompt)
print(response)

## ইন্টারঅ্যাকটিভ চ্যাট সেশন

মডেল কীভাবে প্রতিক্রিয়া জানায় তা দেখতে বিভিন্ন ধরণের প্রম্পট চেষ্টা করুন:


In [ ]:
# Example prompts to try
example_prompts = [
    "What are the benefits of edge AI?",
    "Explain the difference between local and cloud AI inference.",
    "Write a short Python function to calculate the factorial of a number.",
    "What is Microsoft Foundry Local?"
]

for i, prompt in enumerate(example_prompts, 1):
    print(f"\n{'='*50}")
    print(f"Example {i}: {prompt}")
    print(f"{'='*50}")
    
    response = chat_with_model(prompt, max_tokens=200)
    print(response)

## উন্নত ব্যবহার: স্ট্রিমিং প্রতিক্রিয়া

দীর্ঘ প্রতিক্রিয়ার ক্ষেত্রে, স্ট্রিমিং একটি ভালো ব্যবহারকারীর অভিজ্ঞতা প্রদান করতে পারে:


In [ ]:
def chat_with_streaming(prompt, max_tokens=300):
    """Send a chat message with streaming response."""
    try:
        stream = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens,
            stream=True
        )
        
        print("🤖 Assistant (streaming):")
        full_response = ""
        for chunk in stream:
            if chunk.choices[0].delta.content is not None:
                content = chunk.choices[0].delta.content
                print(content, end="", flush=True)
                full_response += content
        print("\n")  # New line after streaming
        return full_response
    except Exception as e:
        return f"Error: {e}"

# Test streaming with a longer prompt
streaming_prompt = "Explain the advantages of running AI models locally on device versus in the cloud. Include privacy, latency, and cost considerations."
print(f"👤 User: {streaming_prompt}\n")
streaming_response = chat_with_streaming(streaming_prompt)

## ত্রুটি পরিচালনা এবং ডায়াগনস্টিকস

চলুন পরিষেবার অবস্থা এবং উপলব্ধ মডেলগুলি পরীক্ষা করি:


In [ ]:
def check_service_health():
    """Check Foundry Local service health."""
    try:
        # Try to list available models
        models_response = client.models.list()
        models = [model.id for model in models_response.data]
        
        print("✅ Service is healthy")
        print(f"📋 Available models: {models}")
        print(f"🎯 Current model: {model}")
        
        if model in models:
            print("✅ Current model is available")
        else:
            print("⚠️ Current model may not be loaded")
            
    except Exception as e:
        print(f"❌ Service check failed: {e}")
        print("\n🔧 Troubleshooting tips:")
        print("1. Make sure Foundry Local is running")
        print("2. Check if a model is loaded: foundry service ps")
        print("3. Verify the endpoint URL is correct")

check_service_health()

## কাস্টম প্রম্পট পরীক্ষা

নিচের সেলে আপনার নিজস্ব প্রম্পট পরীক্ষা করুন:


In [ ]:
# Enter your custom prompt here
custom_prompt = "Write a haiku about artificial intelligence running on edge devices."

print(f"👤 User: {custom_prompt}\n")
custom_response = chat_with_model(custom_prompt, max_tokens=100)
print(f"🤖 Assistant: {custom_response}")

## সারসংক্ষেপ

এই নোটবুকে প্রদর্শিত হয়েছে:

1. **✅ ক্লায়েন্ট সেটআপ**: Foundry Local দিয়ে OpenAI SDK কনফিগার করার পদ্ধতি
2. **✅ বেসিক চ্যাট**: সহজ অনুরোধ-প্রতিক্রিয়া ইন্টারঅ্যাকশন
3. **✅ স্ট্রিমিং**: রিয়েল-টাইম প্রতিক্রিয়া তৈরি
4. **✅ ত্রুটি পরিচালনা**: শক্তিশালী ত্রুটি পরিচালনা এবং ডায়াগনস্টিক
5. **✅ সার্ভিসের স্বাস্থ্য**: মডেলের প্রাপ্যতা এবং সার্ভিসের স্ট্যাটাস পরীক্ষা

### পরবর্তী পদক্ষেপ

- **নমুনা ০২**: Azure OpenAI সমর্থন সহ উন্নত SDK ইন্টিগ্রেশন
- **নমুনা ০৪**: একটি Chainlit চ্যাট অ্যাপ্লিকেশন তৈরি করা
- **নমুনা ০৫**: মাল্টি-এজেন্ট অর্কেস্ট্রেশন সিস্টেম
- **নমুনা ০৬**: বুদ্ধিমান মডেল রাউটিং

### Foundry Local-এর প্রধান সুবিধা

- 🔒 **গোপনীয়তা**: ডেটা আপনার ডিভাইসের বাইরে যায় না
- ⚡ **গতি**: কম লেটেন্সি লোকাল ইনফারেন্স
- 💰 **খরচ**: API ব্যবহারের কোনো খরচ নেই
- 🔌 **অফলাইন**: ইন্টারনেট সংযোগ ছাড়াই কাজ করে
- 🛠️ **সামঞ্জস্যতা**: OpenAI-সামঞ্জস্যপূর্ণ API
